## Trabahlo Prático 1 - Computação Gráfica

#### Classes

In [49]:
from win32verstamp import pad32


class Ponto:
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

class Viewport:
    def __init__(self, vpmin, vpmax):
        self.vpmin = vpmin
        self.vpmax = vpmax

class Window:
    def __init__(self, wmin, wmax):
        self.wmin = wmin
        self.wmax = wmax

class Reta:
    def __init__(self, *pontos):
        self.pontosReta = []
        for ponto in pontos:
            self.pontosReta.append(ponto)

    def __str__(self):
        return 'Reta: ('+','.join(str(e) for e in self.pontosReta)+')'

class Poligono:
    def __init__(self, *pontos):
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)

    def __str__(self):
        return 'Poligono: ('+','.join(str(e) for e in self.pontosPoligono)+')'

In [50]:
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET

class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
        
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        file_menu.add_command(label="Salvar", command=self.salvar_dados)
        
        # Frame principal para conter canvas e minimapa
        self.frame_principal = tk.Frame(root)
        self.frame_principal.pack(fill="both", expand=True)
        
        # Desenhar as viewports iniciais
        # Canvas da Viewport principal
        self.canvas = tk.Canvas(self.frame_principal, width=800, height=600, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
        
        # Canvas do minimapa na lateral direita
        # Utiliza um tamanho fixo para o minimapa
        self.minimap = tk.Canvas(self.frame_principal, width=150, height=120, bg="lightgrey")
        self.minimap.pack(side="right", padx=10, pady=10)
        
        self.viewport_minimapa = Viewport(Ponto(0,0), Ponto(160,120))
        self.window_minimapa = Window(Ponto(0,0), Ponto(0,0))
        
        self.caixa_magica = Poligono(Ponto(0,0), Ponto(0,0))
        
        self.root.bind("<Up>", lambda event: self.mover_window("up"))
        self.root.bind("<Down>", lambda event: self.mover_window("down"))
        self.root.bind("<Left>", lambda event: self.mover_window("left"))
        self.root.bind("<Right>", lambda event: self.mover_window("right"))
    
    def mover_window(self, direcao):
        # Define o valor do deslocamento
        deslocamento = 1

        # Move a window conforme a direção
        if direcao == "up":
            self.window.wmin.y += deslocamento
            self.window.wmax.y += deslocamento
        elif direcao == "down":
            self.window.wmin.y -= deslocamento
            self.window.wmax.y -= deslocamento
        elif direcao == "left":
            self.window.wmin.x -= deslocamento
            self.window.wmax.x -= deslocamento
        elif direcao == "right":
            self.window.wmin.x += deslocamento
            self.window.wmax.x += deslocamento

        # Recalcula e redesenha a viewport com a nova posição da window
        self.desenhar_viewport()   
        self.desenhar_minimapa()
    
    # Realiza a abertura do arquivo e chama a função para carregar seus dados
    def abrir_arquivo(self):
        caminho = filedialog.askopenfilename(filetypes=[("Arquivo XML", "*.xml")])
        if caminho:
            self.carregar_arquivo(caminho)
            self.desenhar_viewport()
            self.desenhar_minimapa()
        pass

    # Realiza a leitura dos dados do arquivo XML selecionado
    def carregar_arquivo(self, caminho):
        tree = ET.parse(caminho)
        root = tree.getroot()
        
        # Realiza a leitura dos dados da Viewport
        viewport = root.find('viewport')
        if viewport is not None:
            vpmin = viewport.find('vpmin')
            vpmax = viewport.find('vpmax')
            ponto_vpmin = Ponto(vpmin.attrib['x'], vpmin.attrib['y'])
            ponto_vpmax = Ponto(vpmax.attrib['x'], vpmax.attrib['y'])
            self.viewport = Viewport(ponto_vpmin, ponto_vpmax)
        
        # Realiza a leitura dos dados da window 
        window = root.find('window')
        if window is not None:
            wmin = window.find('wmin')
            wmax = window.find('wmax')
            ponto_wmin = Ponto(wmin.attrib['x'], wmin.attrib['y'])
            ponto_wmax = Ponto(wmax.attrib['x'], wmax.attrib['y'])
            self.window = Window(ponto_wmin, ponto_wmax)
            
        # Realiza a leitura dos pontos
        self.pontos = []
        for ponto in root.findall('ponto'):
            p = Ponto(ponto.attrib['x'], ponto.attrib['y'])
            self.pontos.append(p)
            
        # Realiza a leitura dos retas
        pontosReta = [] #vetor de pontos de cada reta
        self.retas = [] #vetor de retas
        for reta in root.findall('reta'):
            pontosReta.clear()
            for ponto in reta:
                x = ponto.attrib['x']
                y = ponto.attrib['y']
                pontosReta.append(Ponto(x, y))
            r = Reta(*pontosReta)
            self.retas.append(r)
        
        # Realiza a leitura dos poligonos
        pontosPoligono = []
        self.poligonos = []
        for poligono in root.findall('poligono'):
            pontosPoligono.clear()
            for ponto in poligono:
                x = ponto.attrib['x']
                y = ponto.attrib['y']
                pontosPoligono.append(Ponto(x, y))
            p = Poligono(*pontosPoligono)
            self.poligonos.append(p)
        
    # Aplicar a transformada de viewport a um ponto e retornar o ponto calculado
    def window2viewport(self, ponto):
        x_viewport = ((ponto.x - self.window.wmin.x) / (self.window.wmax.x - self.window.wmin.x)) * (self.viewport.vpmax.x - self.viewport.vpmin.x)
        y_viewport = (1-((ponto.y - self.window.wmin.y) / (self.window.wmax.y - self.window.wmin.y))) * (self.viewport.vpmax.y - self.viewport.vpmin.y)
        return Ponto(x_viewport, y_viewport)
    
    def window_minimap_viewport(self, ponto, window, viewport):
        x_viewport = ((ponto.x - window.wmin.x) / (window.wmax.x - window.wmin.x)) * (viewport.vpmax.x - viewport.vpmin.x)
        y_viewport = (1-((ponto.y - window.wmin.y) / (window.wmax.y - window.wmin.y))) * (viewport.vpmax.y - viewport.vpmin.y)
        return Ponto(x_viewport, y_viewport)

    # Redensenhar a viewport a partir dos dados obtidos do XML e das movimentações
    def desenhar_viewport(self):
        # Apaga o canvas anterior
        if hasattr(self, 'canvas'):
            self.canvas.destroy()
        
        largura = self.viewport.vpmax.x - self.viewport.vpmin.x
        altura = self.viewport.vpmax.y - self.viewport.vpmin.y
        self.canvas = tk.Canvas(self.frame_principal, width=largura, height=altura, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
        
        # Desenha os pontos
        for ponto in self.pontos:
            ponto_vp = self.window2viewport(ponto)
            self.canvas.create_oval(ponto_vp.x - 3, ponto_vp.y - 3, ponto_vp.x + 3, ponto_vp.y + 3, fill="black")
            
        # Desenha as retas
        for reta in self.retas:
            pontoInicial = self.window2viewport(reta.pontosReta[0])
            pontoFinal = self.window2viewport(reta.pontosReta[1])
            self.canvas.create_line(pontoInicial.x, pontoInicial.y, pontoFinal.x, pontoFinal.y, fill="blue", width=2)
        
        # Desenha os poligonos
        for poligono in self.poligonos:
            pontos_vp = [self.window2viewport(ponto) for ponto in poligono.pontosPoligono]
            
            coordenadas = []
            for ponto in pontos_vp:
                coordenadas.append(ponto.x)
                coordenadas.append(ponto.y)
                
            self.canvas.create_polygon(coordenadas, outline="red", fill="", width=2)
    
    # Desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        # Apaga o minimapa anterior
        if hasattr(self, 'minimap'):
            self.minimap.destroy()
        
        self.minimap = tk.Canvas(self.frame_principal, width=160, height=120, bg="lightgrey") # razão de aspecto 4:3
        self.minimap.pack(side="right", padx=10, pady=10)    
        
        # Calculos de altura e largura da window
        largura_win = self.window.wmax.x - self.window.wmin.x
        altura_win = self.window.wmax.y - self.window.wmin.y
        
        self.window_minimapa.wmin.x = self.window.wmin.x - largura_win * 1
        self.window_minimapa.wmin.y = self.window.wmin.y - altura_win * 1
        self.window_minimapa.wmax.x = self.window.wmax.x + largura_win * 1
        self.window_minimapa.wmax.y = self.window.wmax.y + altura_win * 1
        
        # Desenha os pontos
        for ponto in self.pontos:
            ponto_vp = self.window_minimap_viewport(ponto, self.window_minimapa, self.viewport_minimapa)
            self.minimap.create_oval(ponto_vp.x - 1, ponto_vp.y - 1, ponto_vp.x + 1, ponto_vp.y + 1, fill="black")
        
        # Desenha as retas
        for reta in self.retas:
            pontoInicial = self.window_minimap_viewport(reta.pontosReta[0], self.window_minimapa, self.viewport_minimapa)
            pontoFinal = self.window_minimap_viewport(reta.pontosReta[1], self.window_minimapa, self.viewport_minimapa)
            self.minimap.create_line(pontoInicial.x, pontoInicial.y, pontoFinal.x, pontoFinal.y, fill="blue", width=1)
        
        # Desenha os poligonos
        for poligono in self.poligonos:
            pontos_vp = [self.window_minimap_viewport(ponto, self.window_minimapa, self.viewport_minimapa) for ponto in poligono.pontosPoligono]
            
            coordenadas = []
            for ponto in pontos_vp:
                coordenadas.append(ponto.x)
                coordenadas.append(ponto.y)
                
            self.minimap.create_polygon(coordenadas, outline="red", fill="", width=1)
        
        p1 = Ponto(self.window.wmin.x, self.window.wmin.y)
        p2 = Ponto(self.window.wmax.x, self.window.wmax.y)
        p3 = Ponto(self.window.wmin.x + largura_win,self.window.wmin.y) 
        p4 = Ponto(self.window.wmin.x, self.window.wmin.y + altura_win)
        
        self.caixa_magica = []
        pontosPoligono = []
        pontosPoligono.append(p1)
        pontosPoligono.append(p3)
        pontosPoligono.append(p2)
        pontosPoligono.append(p4)
        self.caixa_magica = Poligono(*pontosPoligono)
        
        # Desenho da Caixa Magica
        pontos_cm = [self.window_minimap_viewport(ponto, self.window_minimapa, self.viewport_minimapa) for ponto in self.caixa_magica.pontosPoligono]
        coordenadas = []
        for ponto in pontos_cm:
            coordenadas.append(ponto.x)
            coordenadas.append(ponto.y)
        self.minimap.create_polygon(coordenadas, outline="purple", fill="", width=1)
        
    def salvar_dados(self):
        print("Função acessada")
  
if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()
